In [48]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from itertools import combinations
from decimal import Decimal

pd.set_option('display.max_columns', None)

In [3]:
def caminho_absoluto(caminho_relativo_com_barras_normais):
    
    caminho_base = os.getcwd()

    caminho_absoluto = os.path.join(caminho_base, caminho_relativo_com_barras_normais)

    return caminho_absoluto

In [4]:
situacao_final = pd.read_excel(caminho_absoluto('data/tratamento_curva_abc/dados_tratados/situacao_final.xlsx')).set_index('Ordem')
local_frac = pd.read_excel(caminho_absoluto('data/analise_curva_abc/local/datasets/local_apanha_frac.xlsx'))

In [5]:
situacao_final.iloc[[0]]

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,Dias Pedido Geral,Ativ.Ressupr.Frac,Ativ.Ressupr.Cx,Ativ.Ressupr.Geral,Média por dia frac,Média por dia cx,Média por dia geral,Estoque Armaz.,Ender.Cx.Fechada,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Fracionado,Tipo,Qtde Desvio Picking,local
Ordem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,106011,NARIDRIN 12 HORAS FR C/30ML,A,A,A,279389.0,6507.0,930089.0,271.0,251.0,274.0,1490.0,164.0,1490.0,1030.9557,25.9243,3394.4854,3691,016-037-01-02,14,2,100,172,500,Não,29.913,Ponta De G,3236.0,pallet


In [6]:
local_frac.iloc[[0]]

,Ender.Fracionado,modulo,local
0,29.011,1,AB


In [127]:
local_flowrack = 'AA'
numero_modulos = 5

selecao_local_flowrack = (local_frac['local'] == local_flowrack)
local_flowrakc_selecionado = local_frac[selecao_local_flowrack]
enderecos_local_flowrack_selecionado = local_flowrakc_selecionado['Ender.Fracionado']
enderecos_local_flowrack_selecionado

corredores_por_modulo = {1 : ['29', '28'],
                         2 : ['27', '26'],
                         3 : ['25', '24'],
                         4 : ['23', '22'],
                         5 : ['21', '20'],
                         6 : ['19', '18'],
                        }

def selecionar_florack(corredores):    
   selecao_flowrack_corredordes = ((situacao_final['Tipo'] == 'Flowrack') & \
                              (situacao_final['Ender.Fracionado'].astype(str).str.startswith(corredores[0]) | \
                              situacao_final['Ender.Fracionado'].astype(str).str.startswith(corredores[1])
                              )
                           )

   flowrack_corredores = situacao_final[selecao_flowrack_corredordes]
   return flowrack_corredores

def saida_flowrack_no_modulo_pela_classe(modulo):

    flowrack_do_modulo = selecionar_florack(corredores_por_modulo[modulo])

    selecao_flowrack_modulo_X_classe = flowrack_do_modulo['Ender.Fracionado'].isin(enderecos_local_flowrack_selecionado)

    flowrack_modulo_X_classe = flowrack_do_modulo[selecao_flowrack_modulo_X_classe]

    saida_flowrack_modulo_classe = flowrack_modulo_X_classe['Qtde Venda Frac'].tolist()

    return saida_flowrack_modulo_classe

In [139]:
[str(corredores_por_modulo[6][0]), str(corredores_por_modulo[6][1])]

['19', '18']

In [142]:
saida_dos_modulos = [saida_flowrack_no_modulo_pela_classe(1),
                    saida_flowrack_no_modulo_pela_classe(2),
                    saida_flowrack_no_modulo_pela_classe(3),
                    saida_flowrack_no_modulo_pela_classe(4),
                    saida_flowrack_no_modulo_pela_classe(5),
                    saida_flowrack_no_modulo_pela_classe(6),
                    ]

saida_dos_modulos = saida_dos_modulos[:numero_modulos]

In [143]:
soma_total = 0
for lista in saida_dos_modulos:
    for valor in lista:
        soma_total += valor

print(soma_total)

media_saida_modulo = round(soma_total / len(saida_dos_modulos), 1)
media_saida_modulo

2162417.0


432483.4

In [146]:
saidas_list = []
for i in range(len(saida_dos_modulos)):
    sum_lista_i = round(sum(saida_dos_modulos[i]), 1)
    saida_dif_sum_lista_i = round((sum_lista_i - media_saida_modulo), 1)
    saidas_list.append(Decimal(str(saida_dif_sum_lista_i)))

In [147]:
saidas_list

[Decimal('2384.6'),
 Decimal('-217630.4'),
 Decimal('-32600.4'),
 Decimal('102903.6'),
 Decimal('144942.6')]

In [149]:
sum(saidas_list)

Decimal('0.0')

In [152]:
def encontrar_combinacao(lista, alvo):
    for i in range(1, len(lista) + 1):
        for comb in combinations(lista, i):
            if sum(comb) == alvo:
                return list(comb)
    return None

# Lista de números
modulo = saida_dos_modulos[0]

# Número alvo
total_de_saida_desejada = 74465

# Encontrar combinação
comb = encontrar_combinacao(modulo, total_de_saida_desejada)

if comb:
    print(f"Combinação encontrada: {comb}")
else:
    print("Não foi possível encontrar uma combinação.")

Combinação encontrada: [32687.0, 18102.0, 16043.0, 6159.0, 1474.0]
